In [ ]:
from pathlib import Path

from afft.io import read_file
from afft.utils.log import logger

In [ ]:
from afft.services.sirius_messages import (
    ImageCaptureMessage,
    SeabirdCTDMessage,
    AanderaaCTDMessage,
    EcopuckMessage,
    ParosciPressureMessage,
    TeledyneDVLMessage,
    LQModemMessage,
    EvologicsModemMessage,
    MicronSonarMessage,
    OASonarMessage,
    BatteryMessage,
    ThrusterMessage,
    MESSAGE_TYPE_TO_PARSER,
)

from afft.services.sirius_messages import AuvMessageHeader, parse_message_header

### Parse all messages

In [ ]:
paths: dict = dict()

paths["datadir"] = Path(
    "/media/martin/pcie_01/acfr_revisits_processed/acfr_merged_message_files"
)
paths["messages"] = paths["datadir"] / Path("qd61g27j_20170523_040815_messages.txt")

lines: list[str] = read_file(paths["messages"]).unwrap()

# Mapping from topic to message type
TOPIC_TO_TYPE: dict[str, type] = {
    "VIS": ImageCaptureMessage,
    "SEABIRD": SeabirdCTDMessage,
    "AANDERAA_4317": AanderaaCTDMessage,
    "ECOPUCK": EcopuckMessage,
    "PAROSCI": ParosciPressureMessage,
    "RDI": TeledyneDVLMessage,
    "LQMODEM": LQModemMessage,
    "EVOLOGICS_FIX": EvologicsModemMessage,
    "MICRON": MicronSonarMessage,
    "MICRON_RETURNS": MicronSonarMessage,
    "OAS": OASonarMessage,
    "BATT": BatteryMessage,
    "BATT0": BatteryMessage,
    "BATT1": BatteryMessage,
    "BATT2": BatteryMessage,
    "THR_PORT": ThrusterMessage,
    "THR_STBD": ThrusterMessage,
    "THR_VERT": ThrusterMessage,
}


def parse_messages(lines: list[str]) -> dict:
    """TODO"""
    parsed_messages: dict[type, list] = dict()
    for line in lines:
        header: AuvMessageHeader = parse_message_header(line).unwrap()

        if header.topic not in TOPIC_TO_TYPE:
            continue

        message_type: type = TOPIC_TO_TYPE[header.topic]
        message: type = MESSAGE_TYPE_TO_PARSER[message_type](line).unwrap()

        if message_type not in parsed_messages:
            parsed_messages[message_type] = list()

        parsed_messages[message_type].append(message)

    return parsed_messages


parsed_messages: dict = parse_messages(lines)

for message_type, messages in parsed_messages.items():
    logger.info(f"{message_type}: {len(messages)}")

for message_type, messages in parsed_messages.items():
    logger.info("")
    logger.info(f"{messages[0]}")